In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- 1. Define Sets and Mappings ---
# These sets will be populated dynamically from the CSV headers/indices
# Mappings are crucial for linking string names from CSVs to numerical IDs for Gurobi

# Manufacturers (from fixedcost_manu_to_dc.csv columns)
# DCs (from fixedcost_manu_to_dc.csv rows and fixedcost_dc_to_store.csv columns)
# Stores (from store_demand_data.csv rows and fixedcost_dc_to_store.csv rows)
# Products (from store_demand_data.csv columns and variablecost_manu_to_dc_prodX.csv filenames)

# Manually define mappings based on expected CSV content
# This ensures consistent IDs even if a CSV is missing a specific entry.
mfg_name_to_id = {'CAM': 1, 'LEV': 2, 'LETH': 3}
id_to_mfg_name = {v: k for k, v in mfg_name_to_id.items()}
MANUFACTURERS = list(mfg_name_to_id.keys()) # Use names for sets for clarity in loops

dc_name_to_id = {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}
id_to_dc_name = {v: k for k, v in dc_name_to_id.items()}
DCS = list(dc_name_to_id.keys())

store_name_to_id = {'MONT': 1, 'NEW': 2, 'KEL': 3}
id_to_store_name = {v: k for k, v in store_name_to_id.items()}
STORES = list(store_name_to_id.keys())

product_name_to_id = {'Product 1': 1, 'Product 2': 2}
id_to_product_name = {v: k for k, v in product_name_to_id.items()}
PRODUCTS = list(product_name_to_id.keys())


print("--- Sets Defined ---")
print(f"Products: {PRODUCTS}")
print(f"Manufacturers: {MANUFACTURERS}")
print(f"Distribution Centers: {DCS}")
print(f"Stores: {STORES}\n")

print("--- Mappings ---")
print(f"Manufacturer Mappings: {mfg_name_to_id}")
print(f"DC Mappings: {dc_name_to_id}")
print(f"Store Mappings: {store_name_to_id}")
print(f"Product Mappings: {product_name_to_id}\n")


# --- 2. Load Data Parameters from CSVs ---

# Production Cost (c_ij): Cost to produce item 'i' at manufacturer 'j'
df_production_cost = pd.read_csv('cost_mfg_product.csv')
production_cost = {}
for index, row in df_production_cost.iterrows():
    mfg_id = mfg_name_to_id.get(row['MFG'])
    for col in ['Product 1', 'Product 2']:
        product_id = product_name_to_id.get(col)
        if mfg_id is not None and product_id is not None:
            production_cost[(product_id, mfg_id)] = row[col]


# Manufacturer Capacity (cap_j): Maximum total production for each manufacturer
# NOTE: This data was not in a provided CSV, so it remains hardcoded.
manufacturer_capacity = {
    mfg_name_to_id['CAM']: 151000,
    mfg_name_to_id['LEV']: 91000,
    mfg_name_to_id['LETH']: 61000,
}

# DC Capacity (cap_k): Maximum total throughput for each distribution center
# NOTE: This data was not in a provided CSV, so it remains hardcoded.
dc_capacity = {
    dc_name_to_id['CORN']: 111000,
    dc_name_to_id['SUR']: 95000,
    dc_name_to_id['CAL']: 63000,
    dc_name_to_id['VAU']: 48000,
}

# Store Capacity (cap_e): Maximum total throughput for each store
# NOTE: This data was not in a provided CSV, so it remains hardcoded.
store_capacity = {
    store_name_to_id['MONT']: 75000,
    store_name_to_id['NEW']: 77000,
    store_name_to_id['KEL']: 61000,
}


# Fixed Cost Manufacturer-DC (fixed_manu_dc_jk)
df_fixed_manu_dc = pd.read_csv('fixedcost_manu_to_dc.csv')
fixed_manu_dc_cost = {}
# Assuming the first column is unnamed and contains DC names
df_fixed_manu_dc.rename(columns={df_fixed_manu_dc.columns[0]: 'DC_Name'}, inplace=True)
for index, row in df_fixed_manu_dc.iterrows():
    dc_id = dc_name_to_id.get(row['DC_Name'])
    for col_name in df_fixed_manu_dc.columns[1:]:
        mfg_id = mfg_name_to_id.get(col_name)
        if mfg_id is not None and dc_id is not None:
            fixed_manu_dc_cost[(mfg_id, dc_id)] = row[col_name]


# Variable Cost Manufacturer-DC (var_manu_dc_ijk)
var_manu_dc_cost = {}
for prod_name, prod_id in product_name_to_id.items():
    prod_file_suffix = 'prod1.csv' if prod_name == 'Product 1' else 'prod2.csv'
    df_var_manu_dc = pd.read_csv(f'variablecost_manu_to_dc_{prod_file_suffix}')
    df_var_manu_dc.rename(columns={df_var_manu_dc.columns[0]: 'DC_Name'}, inplace=True)
    for index, row in df_var_manu_dc.iterrows():
        dc_id = dc_name_to_id.get(row['DC_Name'])
        for col_name in df_var_manu_dc.columns[1:]:
            mfg_id = mfg_name_to_id.get(col_name)
            if dc_id is not None and mfg_id is not None:
                var_manu_dc_cost[(prod_id, mfg_id, dc_id)] = row[col_name]


# Fixed Cost DC-Store (fixed_dc_store_ke)
df_fixed_dc_store = pd.read_csv('fixedcost_dc_to_store.csv')
fixed_dc_store_cost = {}
df_fixed_dc_store.rename(columns={df_fixed_dc_store.columns[0]: 'Store_Name'}, inplace=True)
for index, row in df_fixed_dc_store.iterrows():
    store_id = store_name_to_id.get(row['Store_Name'])
    for col_name in df_fixed_dc_store.columns[1:]:
        dc_id = dc_name_to_id.get(col_name)
        if store_id is not None and dc_id is not None:
            fixed_dc_store_cost[(dc_id, store_id)] = row[col_name]


# Variable Cost DC-Store (var_dc_store_ike)
var_dc_store_cost = {}
for prod_name, prod_id in product_name_to_id.items():
    prod_file_suffix = 'prod1.csv' if prod_name == 'Product 1' else 'prod2.csv'
    df_var_dc_store = pd.read_csv(f'variablecost_dc_to_store_{prod_file_suffix}')
    df_var_dc_store.rename(columns={df_var_dc_store.columns[0]: 'Store_Name'}, inplace=True)
    for index, row in df_var_dc_store.iterrows():
        store_id = store_name_to_id.get(row['Store_Name'])
        for col_name in df_var_dc_store.columns[1:]:
            dc_id = dc_name_to_id.get(col_name)
            if store_id is not None and dc_id is not None:
                var_dc_store_cost[(prod_id, dc_id, store_id)] = row[col_name]


# Store Demand (demand_ie)
df_store_demand = pd.read_csv('store_demand_data.csv')
store_demand = {}
for index, row in df_store_demand.iterrows():
    store_id = store_name_to_id.get(row['Store_ID'])
    if store_id is not None:
        store_demand[(product_name_to_id['Product 1'], store_id)] = row['Product 1 Demand']
        store_demand[(product_name_to_id['Product 2'], store_id)] = row['Product 2 Demand']


# A sufficiently large number for Big-M constraints
# This links binary variables (route open/closed) to continuous flow variables.
BIG_M = 1000000

print("--- Data Parameters Loaded from CSVs (and hardcoded where necessary) --- \n")


# --- 3. Create Gurobi Model ---
model = gp.Model("SimplifiedSupplyChain")

# --- 4. Define Decision Variables ---

# x_ij: Quantity of product 'i' produced by manufacturer 'j'
x_ij = model.addVars(product_name_to_id.values(), mfg_name_to_id.values(), name="production", vtype=GRB.CONTINUOUS, lb=0.0)

# trans_ijk: Quantity of product 'i' shipped from manufacturer 'j' to DC 'k'
trans_ijk = model.addVars(product_name_to_id.values(), mfg_name_to_id.values(), dc_name_to_id.values(), name="manu_to_dc_flow", vtype=GRB.CONTINUOUS, lb=0.0)

# trans_ike: Quantity of product 'i' shipped from DC 'k' to store 'e'
trans_ike = model.addVars(product_name_to_id.values(), dc_name_to_id.values(), store_name_to_id.values(), name="dc_to_store_flow", vtype=GRB.CONTINUOUS, lb=0.0)

# y_jk: Binary variable, 1 if route from manufacturer 'j' to DC 'k' is open, 0 otherwise
y_jk = model.addVars(mfg_name_to_id.values(), dc_name_to_id.values(), name="manu_dc_route_open", vtype=GRB.BINARY)

# z_ke: Binary variable, 1 if route from DC 'k' to store 'e' is open, 0 otherwise
z_ke = model.addVars(dc_name_to_id.values(), store_name_to_id.values(), name="dc_store_route_open", vtype=GRB.BINARY)

print("--- Decision Variables Defined --- \n")

# --- 5. Set Objective Function ---
# Objective: Minimize Total Cost
# Total Cost = Production Cost + Fixed Manu-DC Cost + Fixed DC-Store Cost +
#              Variable Manu-DC Transport Cost + Variable DC-Store Transport Cost

# Component 1: Production Cost
# Sum over all products (i) and manufacturers (j): x_ij * production_cost[i,j]
obj_production_cost = gp.quicksum(
    x_ij[prod_id, mfg_id] * production_cost[prod_id, mfg_id]
    for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values()
)

# Component 2: Fixed Cost for Manufacturer-DC routes
# Sum over all manufacturers (j) and DCs (k): fixed_manu_dc_cost[j,k] * y_jk[j,k]
obj_fixed_manu_dc_cost = gp.quicksum(
    fixed_manu_dc_cost[mfg_id, dc_id] * y_jk[mfg_id, dc_id]
    for mfg_id in mfg_name_to_id.values() for dc_id in dc_name_to_id.values()
)

# Component 3: Fixed Cost for DC-Store routes
# Sum over all DCs (k) and stores (e): fixed_dc_store_cost[k,e] * z_ke[k,e]
obj_fixed_dc_store_cost = gp.quicksum(
    fixed_dc_store_cost[dc_id, store_id] * z_ke[dc_id, store_id]
    for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values()
)

# Component 4: Variable Transportation Cost Manufacturer-DC
# Sum over all products (i), manufacturers (j), and DCs (k):
# var_manu_dc_cost[i,j,k] * trans_ijk[i,j,k]
obj_variable_manu_dc_cost = gp.quicksum(
    var_manu_dc_cost[prod_id, mfg_id, dc_id] * trans_ijk[prod_id, mfg_id, dc_id]
    for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values() for dc_id in dc_name_to_id.values()
)

# Component 5: Variable Transportation Cost DC-Store
# Sum over all products (i), DCs (k), and stores (e):
# var_dc_store_cost[i,k,e] * trans_ike[i,k,e]
obj_variable_dc_store_cost = gp.quicksum(
    var_dc_store_cost[prod_id, dc_id, store_id] * trans_ike[prod_id, dc_id, store_id]
    for prod_id in product_name_to_id.values() for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values()
)

# Set the overall objective to minimize the sum of all cost components
model.setObjective(
    obj_production_cost +
    obj_fixed_manu_dc_cost +
    obj_fixed_dc_store_cost +
    obj_variable_manu_dc_cost +
    obj_variable_dc_store_cost,
    GRB.MINIMIZE
)

print("--- Objective Function Set (Minimize Total Cost) --- \n")

# --- 6. Add Constraints ---

# Constraint 1: Manufacturer Production Capacity
# The total quantity of all products produced by a manufacturer cannot exceed its capacity.
# Sum(i in PRODUCTS) x_ij <= manufacturer_capacity[j] for all j in MANUFACTURERS
for mfg_id in mfg_name_to_id.values():
    model.addConstr(
        gp.quicksum(x_ij[prod_id, mfg_id] for prod_id in product_name_to_id.values()) <= manufacturer_capacity[mfg_id],
        f"ManuCapacity_{id_to_mfg_name[mfg_id]}"
    )

# Constraint 2: Production must be distributed from Manufacturer
# The total amount of product 'i' produced by manufacturer 'j' must be equal to
# the total amount of product 'i' shipped from manufacturer 'j' to all DCs.
# x_ij == Sum(k in DCS) trans_ijk[i,j,k] for all i in PRODUCTS, j in MANUFACTURERS
for prod_id in product_name_to_id.values():
    for mfg_id in mfg_name_to_id.values():
        model.addConstr(
            x_ij[prod_id, mfg_id] == gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for dc_id in dc_name_to_id.values()),
            f"ManuOutputFlow_{id_to_product_name[prod_id]}_{id_to_mfg_name[mfg_id]}"
        )

# Constraint 3: DC Inbound Capacity
# The total quantity of all products flowing into a DC cannot exceed its capacity.
# Sum(i in PRODUCTS, j in MANUFACTURERS) trans_ijk <= dc_capacity[k] for all k in DCS
for dc_id in dc_name_to_id.values():
    model.addConstr(
        gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values()) <= dc_capacity[dc_id],
        f"DCInboundCapacity_{id_to_dc_name[dc_id]}"
    )

# Constraint 4: Flow Balance at DC
# For each product 'i' and DC 'k', the total inflow from manufacturers must equal
# the total outflow to stores.
# Sum(j in MANUFACTURERS) trans_ijk == Sum(e in STORES) trans_ike for all i in PRODUCTS, k in DCS
for prod_id in product_name_to_id.values():
    for dc_id in dc_name_to_id.values():
        model.addConstr(
            gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for mfg_id in mfg_name_to_id.values()) == gp.quicksum(trans_ike[prod_id, dc_id, store_id] for store_id in store_name_to_id.values()),
            f"DCFlowBalance_{id_to_product_name[prod_id]}_{id_to_dc_name[dc_id]}"
        )

# Constraint 5: Store Demand Fulfillment
# For each product 'i' and store 'e', the total quantity received from all DCs
# must be at least the store's demand for that product.
# Sum(k in DCS) trans_ike >= store_demand[i,e] for all i in PRODUCTS, e in STORES
for prod_id in product_name_to_id.values():
    for store_id in store_name_to_id.values():
        model.addConstr(
            gp.quicksum(trans_ike[prod_id, dc_id, store_id] for dc_id in dc_name_to_id.values()) >= store_demand[prod_id, store_id],
            f"StoreDemand_{id_to_product_name[prod_id]}_{id_to_store_name[store_id]}"
        )

# Constraint 6: Store Capacity (Inbound)
# The total quantity of all products flowing into a store cannot exceed its capacity.
# Sum(i in PRODUCTS, k in DCS) trans_ike <= store_capacity[e] for all e in STORES
for store_id in store_name_to_id.values():
    model.addConstr(
        gp.quicksum(trans_ike[prod_id, dc_id, store_id] for prod_id in product_name_to_id.values() for dc_id in dc_name_to_id.values()) <= store_capacity[store_id],
        f"StoreCapacityInbound_{id_to_store_name[store_id]}"
    )


# Constraint 7: Linking Manufacturer-DC Flow to Route Activation (Big-M constraint)
# If a manufacturer-DC route (j, k) is not open (y_jk = 0), then no product can flow through it.
# If it is open (y_jk = 1), flow up to BIG_M (a very large number) is allowed.
# trans_ijk <= BIG_M * y_jk[j,k] for all i, j, k
for prod_id in product_name_to_id.values():
    for mfg_id in mfg_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            model.addConstr(
                trans_ijk[prod_id, mfg_id, dc_id] <= BIG_M * y_jk[mfg_id, dc_id],
                f"Link_ManuDC_Flow_{id_to_product_name[prod_id]}_{id_to_mfg_name[mfg_id]}_{id_to_dc_name[dc_id]}"
            )

# Constraint 8: Linking DC-Store Flow to Route Activation (Big-M constraint)
# Similar to Constraint 7, but for DC-Store routes.
# trans_ike <= BIG_M * z_ke[k,e] for all i, k, e
for prod_id in product_name_to_id.values():
    for dc_id in dc_name_to_id.values():
        for store_id in store_name_to_id.values():
            model.addConstr(
                trans_ike[prod_id, dc_id, store_id] <= BIG_M * z_ke[dc_id, store_id],
                f"Link_DCStore_Flow_{id_to_product_name[prod_id]}_{id_to_dc_name[dc_id]}_{id_to_store_name[store_id]}"
            )

print("--- Constraints Added --- \n")

# --- 7. Optimize the Model ---
model.optimize()

# --- 8. Print Results ---
print("\n--- Optimization Results ---")

if model.status == GRB.OPTIMAL:
    print(f"Optimal Total Cost: ${model.objVal:,.2f}\n")

    print("--- Production by Manufacturer (x_ij) ---")
    for prod_id in product_name_to_id.values():
        for mfg_id in mfg_name_to_id.values():
            if x_ij[prod_id, mfg_id].x > 1e-6: # Check for non-zero production
                print(f"  {id_to_product_name[prod_id]} produced by {id_to_mfg_name[mfg_id]}: {x_ij[prod_id, mfg_id].x:,.2f} units")

    print("\n--- Manufacturer-DC Routes Open (y_jk) ---")
    for mfg_id in mfg_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            if y_jk[mfg_id, dc_id].x > 0.5: # Check if binary variable is essentially 1
                print(f"  Route from {id_to_mfg_name[mfg_id]} to {id_to_dc_name[dc_id]} is OPEN")

    print("\n--- Flow from Manufacturers to DCs (trans_ijk) ---")
    for prod_id in product_name_to_id.values():
        for mfg_id in mfg_name_to_id.values():
            for dc_id in dc_name_to_id.values():
                if trans_ijk[prod_id, mfg_id, dc_id].x > 1e-6: # Check for non-zero flow
                    print(f"  {id_to_product_name[prod_id]} from {id_to_mfg_name[mfg_id]} to {id_to_dc_name[dc_id]}: {trans_ijk[prod_id, mfg_id, dc_id].x:,.2f} units")

    print("\n--- DC-Store Routes Open (z_ke) ---")
    for dc_id in dc_name_to_id.values():
        for store_id in store_name_to_id.values():
            if z_ke[dc_id, store_id].x > 0.5: # Check if binary variable is essentially 1
                print(f"  Route from {id_to_dc_name[dc_id]} to {id_to_store_name[store_id]} is OPEN")

    print("\n--- Flow from DCs to Stores (trans_ike) ---")
    for prod_id in product_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            for store_id in store_name_to_id.values():
                if trans_ike[prod_id, dc_id, store_id].x > 1e-6: # Check for non-zero flow
                    print(f"  {id_to_product_name[prod_id]} from {id_to_dc_name[dc_id]} to {id_to_store_name[store_id]}: {trans_ike[prod_id, dc_id, store_id].x:,.2f} units")

elif model.status == GRB.INFEASIBLE:
    print("The model is INFEASIBLE. No solution satisfies all constraints.")
    print("Consider reviewing your data and constraints for contradictions.")
elif model.status == GRB.UNBOUNDED:
    print("The model is UNBOUNDED. The objective can be arbitrarily improved.")
    print("This often means a necessary constraint or bound is missing.")
else:
    print(f"Optimization ended with status: {model.status}")

--- Sets Defined ---
Products: ['Product 1', 'Product 2']
Manufacturers: ['CAM', 'LEV', 'LETH']
Distribution Centers: ['CORN', 'SUR', 'CAL', 'VAU']
Stores: ['MONT', 'NEW', 'KEL']

--- Mappings ---
Manufacturer Mappings: {'CAM': 1, 'LEV': 2, 'LETH': 3}
DC Mappings: {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}
Store Mappings: {'MONT': 1, 'NEW': 2, 'KEL': 3}
Product Mappings: {'Product 1': 1, 'Product 2': 2}

--- Data Parameters Loaded from CSVs (and hardcoded where necessary) --- 

Set parameter Username
Set parameter LicenseID to value 2673696
Academic license - for non-commercial use only - expires 2026-05-30
--- Decision Variables Defined --- 

--- Objective Function Set (Minimize Total Cost) --- 

--- Constraints Added --- 

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with